In [5]:
ANY = '_'  # constante a usar na definição dos padrões de registo

class Table:
    def __init__(self, scheme, key_idx, widths):
        """

        :param scheme: string
        :param key_idx: list
        :param widths: list
        """
        self._scheme = scheme.split(' ') # list[]
        self._key_idx = key_idx #primary key
        self._table = {} #punem continutul aici
        self._widths = widths
      
    

    def add(self,insert):#add value to the table
        """

        :param insert: tuple
        :return: none
        """
        key = tuple(insert[i] for i in self._key_idx)

        self._table[key] =insert


    def delete(self,pattern):# deletes from the table
        """

        :param pattern: tuple
        :return: none
        """
        to_remove=[key for key,row in self._table.items() if all(p==ANY or p== val for p,val in zip(pattern,row)) ]
        """
        to_remove=[]
        for key,row in self._table.items():
            if all(p==ANY or p== val for p,val in zip(pattern,row)) :
                to_remove.append(key)
            """
        for key in to_remove:
            del self._table[key]

        
    def lookup(self,pattern):#serches for the pattern in the table and returns the result
        """

        :param pattern: tuple
        :return: list
        """
        result = [row for row in self._table.values() if all(p==ANY or p==val for p, val in zip(pattern, row))]
        """
        result = []
        for row in self._table.values():
            if all(p == ANY or p == val for p,val in zip(pattern,row)):
                result.append(row)"""
        return result

    def select(self,p):#selects a parameter and return a new table
        """

        :param p: function
        :return: object
        """
        new_table=Table(' '.join(self._scheme),key_idx=self._key_idx,widths=self._widths)
        for row in self._table.values():
            if p(row):
                new_table.add(row)
        return new_table



    def project(self,subscheme, subkey_idx):
        """
        :param subscheme: string
        :param subkey_idx: list
        :return: object
        """
        subscheme = subscheme.split(' ')
        index_col =[self._scheme.index(col) for col in subscheme]
        new_width=[self._widths[i] for i in index_col]
        new_table=Table(' '.join(subscheme),key_idx=subkey_idx,widths=new_width)
        for row in self._table.values():
            project_row=tuple(row[i] for i in index_col)
            new_table.add(project_row)
        return new_table





    def __add__(self,other):# return a new table where the new values/table are concatenated
        """

        :param other: object
        :return: object
        """
        result=Table(' '.join(self._scheme), key_idx=self._key_idx, widths=self._widths)
        for row in self._table.values():
            result.add(row)
        for row in other._table.values():
            result.add(row)
        return result



    def __mul__(self,other):#return a new table where we get the common value
        """

        :param other: object
        :return: object
        """
        result=Table(' '.join(self._scheme), key_idx=self._key_idx, widths=self._widths)
        other_key=set(other._table.keys())

        for key,row in self._table.items():
            if key  in other_key:
                result.add(row)
        return result




    def __sub__(self,other):# return the difference between the tables
        """

        :param other: object
        :return: object
        """
        result=Table(' '.join(self._scheme), key_idx=self._key_idx, widths=self._widths)
        other_key=set(other._table.keys())
        for key,row in self._table.items():
            if key not in other_key:
                result.add(row)
        return result



    def __pow__(self, other):
        # combine the scheme without the duplicate
        new_scheme = []
        for col in self._scheme + other._scheme:
            if col not in new_scheme:
                new_scheme.append(col)

        # Making the new primary keys combined
        key_columns = []
        for i in self._key_idx:
            col = self._scheme[i]
            if col not in key_columns:
                key_columns.append(col)
        for i in other._key_idx:
            col = other._scheme[i]
            if col not in key_columns:
                key_columns.append(col)
        new_key_idx = [new_scheme.index(col) for col in key_columns]

        # the new widths of the table
        new_widths = []
        for col in new_scheme:
            if col in self._scheme:
                new_widths.append(self._widths[self._scheme.index(col)])
            else:
                new_widths.append(other._widths[other._scheme.index(col)])

        # new table
        new_table = Table(' '.join(new_scheme), key_idx=new_key_idx, widths=new_widths)

        # making a natural join -> getting the common columns and their indexes

        common_cols = set(self._scheme) & set(other._scheme)
        self_indexes = [self._scheme.index(col) for col in common_cols]
        other_indexes = [other._scheme.index(col) for col in common_cols]
        #checking each row from both tables
        for row1 in self._table.values():
            for row2 in other._table.values():
                if all(row1[i] == row2[j] for i, j in zip(self_indexes, other_indexes)):
                    merged_row = []
                    for col in new_scheme:
                        if col in self._scheme:
                            merged_row.append(row1[self._scheme.index(col)])
                        else:
                            merged_row.append(row2[other._scheme.index(col)])
                    new_table.add(tuple(merged_row))

        return new_table

    def __iter__(self):
        return iter(self._table.items())

  
  ##### não alterar __repr__ #####
    def __repr__(self):  
        header = ' | '.join((k + '!'*(i in self._key_idx)).ljust(self._widths[i])
                            for i,k in enumerate(self._scheme))
        content = '\n'.join( ' | '.join(str(column).ljust(self._widths[i])
                                        for i,column in enumerate(row))
                            for row in self._table.values())
        return '\n'.join((header+'\n'+content ).split('\n'))

    

In [3]:
##################################################################
######### tabelas exemplos para os testes (não alterar!) #########
##################################################################
def make_db():
    addresses = Table('id num address country', key_idx=[0], widths=[3,5,36,8])
    #[id , num, address, country], id!, widths=[len(id),len(num),len(address),len(country)] reprezinta cate caractere este in fiecare coloana
    addresses.add( (1, 50000, "Campo Grande 016, 1749-016 Lisboa",    'PT') )
    addresses.add( (2, 50000, "Av. da Liberdade 2, 1250-144 Lisboa",  'PT') )
    addresses.add( (3, 50001, "Av. da Republica 12, 1210-54 Lisboa",  'PT') )
    addresses.add( (4, 50002, "Av. Infante Santo 8, 1350-001 Lisboa", 'PT') )
    
    students = Table('num name grad year', key_idx=[0], widths=[5,12,5,4])
    students.add( (50000, 'Ana',   'LEI', 2020) )
    students.add( (50001, 'Rui',   'LEI', 2022) )
    students.add( (50002, 'Pedro', 'LTI', 2022) )
    students.add( (50003, 'Dulce', 'LF',  2021) )
    students.add( (50004, 'Pedro', 'LMA', 2022) )    
    
    return addresses, students

### Test to run:

In [4]:
try:
  addresses, students = make_db()
  print(addresses)
  print()
  print(students)
except:
  pass

id! | num   | address                              | country 
1   | 50000 | Campo Grande 016, 1749-016 Lisboa    | PT      
2   | 50000 | Av. da Liberdade 2, 1250-144 Lisboa  | PT      
3   | 50001 | Av. da Republica 12, 1210-54 Lisboa  | PT      
4   | 50002 | Av. Infante Santo 8, 1350-001 Lisboa | PT      

num!  | name         | grad  | year
50000 | Ana          | LEI   | 2020
50001 | Rui          | LEI   | 2022
50002 | Pedro        | LTI   | 2022
50003 | Dulce        | LF    | 2021
50004 | Pedro        | LMA   | 2022


In [5]:
try:
  addresses, students = make_db()
  students.add( (50005,'Francisca','LEA',2023) )
  students.add( (50006,'Zé','LMOG',2023) )
  print(students)
except:
  pass

num!  | name         | grad  | year
50000 | Ana          | LEI   | 2020
50001 | Rui          | LEI   | 2022
50002 | Pedro        | LTI   | 2022
50003 | Dulce        | LF    | 2021
50004 | Pedro        | LMA   | 2022
50005 | Francisca    | LEA   | 2023
50006 | Zé           | LMOG  | 2023


In [6]:
try:
  addresses, students = make_db()
  students.delete( (ANY,'Pedro',ANY,ANY) )
  print(students)
except:
  pass

num!  | name         | grad  | year
50000 | Ana          | LEI   | 2020
50001 | Rui          | LEI   | 2022
50003 | Dulce        | LF    | 2021


In [7]:
try:
  addresses, students = make_db()
  rows = students.lookup( (ANY,ANY,'LEI',ANY) )
  print(rows)
except:
  pass

[(50000, 'Ana', 'LEI', 2020), (50001, 'Rui', 'LEI', 2022)]


In [8]:
try:
  addresses, students = make_db()

  students2 = Table('num name grad year', key_idx=[0], widths=[5,8,5,4])
  students2.add( (50001, 'Rui', 'LEI', 2022) )
  students2.add( (50005, 'Bernardo', 'LF', 2019) )
  students2.add( (50006, 'Anabela', 'LTI', 2021) )

  print(students + students2)
except:
  pass

num!  | name         | grad  | year
50000 | Ana          | LEI   | 2020
50001 | Rui          | LEI   | 2022
50002 | Pedro        | LTI   | 2022
50003 | Dulce        | LF    | 2021
50004 | Pedro        | LMA   | 2022
50005 | Bernardo     | LF    | 2019
50006 | Anabela      | LTI   | 2021


In [9]:
try:
  addresses, students = make_db()

  students2 = Table('num name grad year', key_idx=[0], widths=[5,8,5,4])
  students2.add( (50001, 'Rui', 'LEI', 2022) )
  students2.add( (50005, 'Bernardo', 'LF', 2019) )
  students2.add( (50006, 'Anabela', 'LTI', 2021) )

  print(students * students2)
except:
  pass

num!  | name         | grad  | year
50001 | Rui          | LEI   | 2022


In [10]:
try:
  addresses, students = make_db()

  students2 = Table('num name grad year', key_idx=[0], widths=[5,8,5,4])
  students2.add( (50001, 'Rui', 'LEI', 2022) )
  students2.add( (50005, 'Bernardo', 'LF', 2019) )
  students2.add( (50006, 'Anabela', 'LTI', 2021) )

  print(students - students2)
except:
  pass

num!  | name         | grad  | year
50000 | Ana          | LEI   | 2020
50002 | Pedro        | LTI   | 2022
50003 | Dulce        | LF    | 2021
50004 | Pedro        | LMA   | 2022


In [11]:
try:
  addresses, students = make_db()

  students2 = students.select(lambda row: len(row[1])==5)
  print(students2)
  print()

  students3 = students.select(lambda row: row[2]=='LEI')
  print(students3)
except:
  pass

num!  | name         | grad  | year
50002 | Pedro        | LTI   | 2022
50003 | Dulce        | LF    | 2021
50004 | Pedro        | LMA   | 2022

num!  | name         | grad  | year
50000 | Ana          | LEI   | 2020
50001 | Rui          | LEI   | 2022


In [12]:
try:
  addresses, students = make_db()

  students2 = students.project("year num grad", [1])
  print(students2)
except:
  pass

year | num!  | grad 
2020 | 50000 | LEI  
2022 | 50001 | LEI  
2022 | 50002 | LTI  
2021 | 50003 | LF   
2022 | 50004 | LMA  


In [6]:
try:
  addresses, students = make_db()

  join = students ** addresses
  print(join)
  print()

  join = addresses ** students
  print(join)
except Exception as e:
  print(e)

num!  | name         | grad  | year | id! | address                              | country 
50000 | Ana          | LEI   | 2020 | 1   | Campo Grande 016, 1749-016 Lisboa    | PT      
50000 | Ana          | LEI   | 2020 | 2   | Av. da Liberdade 2, 1250-144 Lisboa  | PT      
50001 | Rui          | LEI   | 2022 | 3   | Av. da Republica 12, 1210-54 Lisboa  | PT      
50002 | Pedro        | LTI   | 2022 | 4   | Av. Infante Santo 8, 1350-001 Lisboa | PT      

id! | num!  | address                              | country  | name         | grad  | year
1   | 50000 | Campo Grande 016, 1749-016 Lisboa    | PT       | Ana          | LEI   | 2020
2   | 50000 | Av. da Liberdade 2, 1250-144 Lisboa  | PT       | Ana          | LEI   | 2020
3   | 50001 | Av. da Republica 12, 1210-54 Lisboa  | PT       | Rui          | LEI   | 2022
4   | 50002 | Av. Infante Santo 8, 1350-001 Lisboa | PT       | Pedro        | LTI   | 2022


In [7]:
try:
  addresses, students = make_db()

  for register in students:
    print(*register)
  print()

  for address in addresses:
    print(*address)
  print()

  for register in students ** addresses:
    print(*register)
except Exception as e:
  print(e)

(50000,) (50000, 'Ana', 'LEI', 2020)
(50001,) (50001, 'Rui', 'LEI', 2022)
(50002,) (50002, 'Pedro', 'LTI', 2022)
(50003,) (50003, 'Dulce', 'LF', 2021)
(50004,) (50004, 'Pedro', 'LMA', 2022)

(1,) (1, 50000, 'Campo Grande 016, 1749-016 Lisboa', 'PT')
(2,) (2, 50000, 'Av. da Liberdade 2, 1250-144 Lisboa', 'PT')
(3,) (3, 50001, 'Av. da Republica 12, 1210-54 Lisboa', 'PT')
(4,) (4, 50002, 'Av. Infante Santo 8, 1350-001 Lisboa', 'PT')

(50000, 1) (50000, 'Ana', 'LEI', 2020, 1, 'Campo Grande 016, 1749-016 Lisboa', 'PT')
(50000, 2) (50000, 'Ana', 'LEI', 2020, 2, 'Av. da Liberdade 2, 1250-144 Lisboa', 'PT')
(50001, 3) (50001, 'Rui', 'LEI', 2022, 3, 'Av. da Republica 12, 1210-54 Lisboa', 'PT')
(50002, 4) (50002, 'Pedro', 'LTI', 2022, 4, 'Av. Infante Santo 8, 1350-001 Lisboa', 'PT')


In [19]:
try:
  T1 = Table("a b c", [0], [2,2,2])
  for i in range(9):
    T1.add( (i,i+1,i+3) )
  T2 = Table("a b c", [0], [2,2,2])
  for i in range(6):
    T2.add( (i,2*i,3*i) )
    T2.add( (i,3*i,5*i) )
    T2.add( (i,4*i,6*i) )

  # print(T1); print()
  # print(T2); print()
  print(T1*T2)
except:
  print('exception at * op')

{(2,), (5,), (4,), (1,), (0,), (3,)}
a! | b  | c 
0  | 1  | 3 
1  | 2  | 4 
2  | 3  | 5 
3  | 4  | 6 
4  | 5  | 7 
5  | 6  | 8 


In [20]:
try:
    T2 = Table("a b c", [0,2], [2,2,2])
    for i in range(6):
        T2.add( (i,2*i,3*i) )
        T2.add( (i,3*i,5*i) )
        T2.add( (i,4*i,6*i) )
        T2.add( (i,i+3,2*i) )
    print(T2)
except:
    print('exception at * op')


a! | b  | c!
0  | 3  | 0 
1  | 2  | 3 
1  | 3  | 5 
1  | 4  | 6 
1  | 4  | 2 
2  | 4  | 6 
2  | 6  | 10
2  | 8  | 12
2  | 5  | 4 
3  | 6  | 9 
3  | 9  | 15
3  | 12 | 18
3  | 6  | 6 
4  | 8  | 12
4  | 12 | 20
4  | 16 | 24
4  | 7  | 8 
5  | 10 | 15
5  | 15 | 25
5  | 20 | 30
5  | 8  | 10
